# Malaria Project - Implementation
## Detecting Malaria cells using CNN and TF 2.0


### By:
- Lukas Wagner s0556753
- Laila Westphal s0556469


## Table of Contents:
* [Introduction](#Introduction)
* [Brief discussion about existing kernels](#Brief-discussion-about-existing-kernels)
  * [Kernel 1](#Kernel-1:-Detecting-Malaria-cells-using-Convolutional-Neural-Network)
  * Kernel 2
* [Getting Started](#Getting-Started) 
  * [Imports](#Imports)
  * [Load Data](#LoadData)
* [Preprocess Data](#Preprocess-Data)
  * [Feature Engineering](#Feature-Engineering)
  * [Data Augmentation](#Data-Augmentation)
  * Generate Data and Labels
  * Split Data
* Data-Visualization
* Setup CNN
* Train data
* Evaluate?
* Predictions using test data
* Visualize Accuracy
* Results, Confusion Matrix
* Visualize Results
* Licenses
* Acknowledgements


# Introduction

# Brief discussion about existing kernels

In this section two kernels from the kaggle website dealing with the malaria dataset will be analized regarding data augmentation, data-split, cnn-architecture, used activation functions and obtained results (accuracy ...).
They are being used as a guideline for our own implementation.

## Kernel 1: Detecting Malaria cells using Convolutional Neural Network
### By: Kushal Mahindrakar
Source:https://www.kaggle.com/kushal1996/detecting-malaria-cnn

#### Data Augmentation:
- Done using the Python Image Libary (PIL) that provides support for opening, editing and saving many different data formats
- The quantity of the "Parasitized" data is being quadrupled.
- The quantity of the "Uninfected" data is being tripled.

#### Data-Split
The data in this kernel is being split into training, evaluation and test data:
   - training-data: 80%
   - evaluation-data: 10%
   - test-data: 10%

#### Used CNN-Architecture and activation functions:

| Layer-Type   | Units | Nb_Filters | KernelSize PoolSize | Padding | Activation | Stride |
|--------------|-------|------------|---------------------|---------|------------|--------|
| Input        | _     | _          | _                   | _       | _          | _      |
|  Conv        | _     | 50         | 7x7                 | same    | ReLU       | _      |
| Conv         | _     | 90         | 3x3                 | valid   | ReLU       | _      |
| Conv         | _     | 10         | 5x5                 | same    | ReLU       | _      |
| Pool         | _     | _          | 2x2                 | _       | _          | 2      |
| Conv         | _     | 5          | 3x3                 | same    | ReLU       | _      |
| Pool         | _     | _          | 2x2                 | same    | _          | 2      |
| Pool_flatten | _     | _          | _                   | _       | _          | _      |
| FC           | 2000  | _          | _                   | _       | ReLU       | _      |
| FC           | 1000  | _          | _                   | _       | ReLU       | _      |
| FC           | 500   | _          | _                   | _       | ReLU       | _      |
| Logits       | 2     | _          | _                   | _       | _          | _      |

#### Obtained Results:

| _           | precision |   recall | f1-score |  support |
|-------------|-----------|----------|----------|----------|
|           0 |      0.90 |     0.93 |     0.91 |     4223 |
|           1 |      0.95 |     0.92 |     0.93 |     5423 |
|_            | _         | _        | _        | _        |
|   micro avg |      0.92 |     0.92 |     0.92 |     9646 |
|   macro avg |      0.92 |     0.92 |     0.92 |     9646 |
|weighted avg |      0.92 |     0.92 |     0.92 |     9646 |
| _           | _         | _        | _        | _        |
|0.9238026124818578



# Getting Started

## Imports

In [66]:
# Uncomment to install TF 2.0 if you don't have it
#!pip install tensorflow==2.0.0-beta1

In [67]:
# INCLUDE DOWNLOAD POSSIBILITY FOR DATA

In [68]:
# linear algebra
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
# computer vision library, image processing
import cv2
# used for data augmentation
from PIL import Image
from PIL import ImageFilter
# to use operating system dependent functionality
import os

## Load Data

In [69]:
# Loads data by classes/categories (e.g Parasitized, Uninfected)
def loadData(path):
    "__________________________________________"
    "    Args:"
    "        path: path to your data"
    "    return: Dict((class, label): list<data>)"
    "__________________________________________"
    
    clazzes = os.listdir(path)
    clazz_data = dict()
    for clazz, label in zip(clazzes, range(len(clazzes))):
            clazz_data.update({(clazz, label) : (os.listdir(path + "/" + clazz))})
    return clazz_data
        

In [70]:
# Check whether you obtained the right results
check_data = loadData("data/cell_images")
for clazz in check_data:
    print(clazz[0] + " " + str(clazz[1]) + ": " + str(len(check_data.get(clazz))))

Parasitized 0: 13780
Uninfected 1: 13780


# Preprocess Data

In this section the data will be preprocessed in order to fulfil the requirements we have:
- feature engineering: here we will eliminate files that are no images (e.g. my Uninfected and Parasitized folders both contain a temp file which we don'T need and that would cause the program to fail)
- resize the image to be of the input size we want to use
- mode: decide on RGB, Greyscale...

## Feature Engineering

In [71]:
def preprocessData(image_path, h_img, w_img, mode='RGB'):
    "__________________________________________"
    "    Args:"
    "        data_dict: Dict(class: list<data>)"
    "        h_img: height of image"
    "        w_img: width of image"
    "    return: "
    "__________________________________________"
    image = cv2.imread(image_path)
    if image is None:
        return None
    img_as_array = Image.fromarray(image, mode)
    preprocessed_img = img_as_array.resize((h_img,w_img))
    return preprocessed_img
    

## Data Augmentation
- form of regularization
- useful for cells because impossible to predict position they will have
--> find source

In [72]:
#    PIL.Image.FLIP_LEFT_RIGHT
#    PIL.Image.FLIP_TOP_BOTTOM
#    PIL.Image.TRANSPOSE

def augmentData(image, label, rotations=None, mirror=None, blur=False):
    img_data = []
    img_labels = []
    if rotations != None:
        for rot in rotations:
            img_data.append(np.array(image.rotate(rot)))
            img_labels.append(label)
    if mirror != None:
        for mi in mirror:
            img_data.append(np.array(image.transpose(Image.mi)))
            img_labels.append(label)
    if blur is True:
        img_data.append(np.array(ImageFilter.BLUR))
        img_labels.append(label)
    return img_data, img_labels
        
    

# Generate all Data and Labels

In [75]:
def generateData(path, h_img, w_img, mode='RGB', rotations=None, mirror=None, blur=False):
    data = []
    labels = []
    clazz_data = load_data(path)
    for clazz in clazz_data:
        for img in clazz_data.get(clazz):
            img_path = path + "/" + clazz + "/" + img
            prep_img = preprocessData(img_path, h_img, w_img)
            if prep_img is None:
                continue
            data.append(np.array(img))
            labels.append(clazz[1])
            augm_data, augm_label = augmentData(prep_img,clazz[1], rotations, mirror, blur)
            data.extend(augm_data)
            labels.extend(augm_label)
    return data, labels
        
    

In [ ]:
#checking whether data and label genration work properly
d, l = generateData("data/cell_images", 50, 50, rotations=[45,75], blur = True)
print(len(d))
print(len(l))

## Licenses

### Notebook License (CC-BY-SA 4.0)

*The following license applies to the complete notebook, including code cells. It does however not apply to any referenced external media (e.g., images).*

Malaria Project - Implementation<br/>
Detecting Malaria cells using CNN and TF 2.0 <br/>
by Lukas Wagner, Laila Westphal <br/>
is licensed under a [Creative Commons Attribution-ShareAlike 4.0 International License](http://creativecommons.org/licenses/by-sa/4.0/).<br/>


### Code License (MIT)

*The following license only applies to code cells of the notebook.*

Copyright 2019 Lukas Wagner, Laila Westphal

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

## Acknowledgements

